In [ ]:
!pip install geopandas rasterio matplotlib geemap leafmap shapely fiona earthengine-api


In [ ]:
!pip uninstall -y earthengine-api geemap
!pip install earthengine-api==0.1.381 geemap==0.30.0


In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='vaibhavi-tiwari-satellite')
print("✅ Earth Engine initialized successfully!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
data_path = '/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/'


In [ ]:
import os

# If testing locally (small demo folder on GitHub)
if os.path.exists("data/testing_folder"):
    folder = "data/testing_folder"
else:
    # Full dataset path on Google Drive
    folder = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/shapefile"

print(os.listdir(folder))


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Defining shapefile path
shapefile_path = '/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/shapefile/delhi_ncr_region.geojson'

# Load shapefile
delhi_ncr = gpd.read_file(shapefile_path)

# View basic info
print(delhi_ncr.head())


In [ ]:
# Plotting Delhi-NCR shapefile
fig, ax = plt.subplots(figsize=(8,8))
delhi_ncr.plot(ax=ax, color='lightblue', edgecolor='black')
ax.set_title("Delhi-NCR Shapefile Boundary", fontsize=14)
plt.show()


In [ ]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='vaibhavi-tiwari-satellite')


In [ ]:
import geopandas as gpd

airshed_path = '/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/shapefile/delhi_airshed.geojson'
airshed = gpd.read_file(airshed_path)

Map = geemap.Map(center=[28.6, 77.2], zoom=8)
Map.add_basemap('SATELLITE')
Map.add_gdf(airshed, layer_name='Delhi Airshed', style={'color':'red', 'fillOpacity':0})
Map


In [ ]:
import json
import ee

# Loading GeoJSON
with open(airshed_path) as f:
    geojson = json.load(f)

# Converting GeoJSON to EE Feature
roi = ee.FeatureCollection(geojson)
Map.addLayer(roi, {'color': 'yellow'}, 'Airshed Boundary')
Map


In [ ]:
# Sentinel-2 Surface Reflectance (collection 2)
s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate('2025-01-01', '2025-01-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .select(['B4', 'B3', 'B2'])
)

# Median composite
s2_median = s2.median().clip(roi)

Map.addLayer(s2_median, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}, 'Sentinel-2 Jan 2025 RGB')
Map


In [ ]:
# Function to create a grid over the ROI
def create_grid(region, scale):
    # Get pixel lon/lat
    lonlat = ee.Image.pixelLonLat().reproject(crs='EPSG:4326', scale=scale)

    # Create grid visualization (we just use lon/lat bands)
    grid = lonlat.select(['longitude', 'latitude'])

    # Clip grid to region of interest
    grid = grid.clip(region)

    return grid

# Create grid (tile size ≈ 1280 m)
grid_img = create_grid(roi, 1280)

# Add to map for visualization
Map.addLayer(grid_img.randomVisualizer(), {}, 'Grid Tiles')
Map


In [ ]:
# Example: export one clipped tile
first_tile = s2_median.clip(roi.geometry())

task = ee.batch.Export.image.toDrive(
    image=first_tile,
    description='delhi_airshed_sample',
    folder='sentinel_downloads',
    scale=10,   # Sentinel-2 resolution
    region=roi.geometry(),
    maxPixels=1e13
)
task.start()


In [ ]:
import ee
import geemap
import geopandas as gpd

ee.Initialize(project='vaibhavi-tiwari-satellite')

# Loading airshed shapefile
airshed_path = '/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/shapefile/delhi_airshed.geojson'
airshed_gdf = gpd.read_file(airshed_path)

# Convert to Earth Engine FeatureCollection
roi = geemap.geopandas_to_ee(airshed_gdf)


In [ ]:
# Load Sentinel-2 SR collection
s2_sr = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(roi)
    .filterDate('2025-01-01', '2025-01-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
)

print("Number of images found:", s2_sr.size().getInfo())


In [ ]:
s2_median = s2_sr.median().clip(roi)


In [ ]:
Map = geemap.Map(center=[28.6, 77.2], zoom=8)
Map.add_basemap('SATELLITE')

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

Map.addLayer(s2_median, vis_params, 'Sentinel-2 Composite')
Map.addLayer(roi, {'color': 'red'}, 'Delhi Airshed')
Map


In [ ]:
import ee, geemap
import math

# Function to create a grid over the ROI (1280m × 1280m)
def create_grid(region, grid_size):
    """
    Create a rectangular grid covering the region, where each cell is grid_size meters.
    """
    # Get bounding box
    bounds = region.geometry().bounds()
    coords = ee.List(ee.List(bounds.coordinates()).get(0))

    # Extract lon/lat arrays
    lon_list = coords.map(lambda p: ee.Number(ee.List(p).get(0)))
    lat_list = coords.map(lambda p: ee.Number(ee.List(p).get(1)))

    lon_min = ee.Number(lon_list.reduce(ee.Reducer.min()))
    lon_max = ee.Number(lon_list.reduce(ee.Reducer.max()))
    lat_min = ee.Number(lat_list.reduce(ee.Reducer.min()))
    lat_max = ee.Number(lat_list.reduce(ee.Reducer.max()))

    # Convert grid size from meters to degrees
    step_deg = grid_size / 111320.0

    lon_steps = ee.List.sequence(lon_min, lon_max, step_deg)
    lat_steps = ee.List.sequence(lat_min, lat_max, step_deg)

    def make_cells(lat):
        def make_cell(lon):
            rect = ee.Geometry.Rectangle(
                [lon, lat, ee.Number(lon).add(step_deg), ee.Number(lat).add(step_deg)]
            )
            return ee.Feature(rect)
        return lon_steps.map(make_cell)

    # Flatten the 2D list of tiles
    grid_list = lat_steps.map(make_cells).flatten()

    # Convert to FeatureCollection and clip to region
    grid = ee.FeatureCollection(grid_list).filterBounds(region)
    return grid

# Create the grid (1280 m)
grid_fc = create_grid(roi, 1280)

# Print tile count
print("Total tiles generated:", grid_fc.size().getInfo())

# Visualize
Map = geemap.Map(center=[28.6, 77.2], zoom=9)
Map.addLayer(grid_fc, {'color': 'yellow'}, '1280m Grid')
Map.addLayer(roi, {'color': 'red'}, 'Delhi Airshed')
Map


In [ ]:
import time
import ee

# ✅ Export folder inside Google Drive
export_folder = "Export_Satellite_Images"

# ✅ Function to export each tile
def export_tile(tile_feature):
    tile = ee.Feature(tile_feature)
    geom = tile.geometry()

    # Compute centroid for filename
    centroid = geom.centroid()
    lon = ee.Number(centroid.coordinates().get(0)).format('%.4f')
    lat = ee.Number(centroid.coordinates().get(1)).format('%.4f')
    name = ee.String('tile_').cat(lat).cat('_').cat(lon)

    # Clip composite
    clipped = s2_median.clip(geom)

    # Start export task
    task = ee.batch.Export.image.toDrive(
        image=clipped,
        description=name.getInfo(),
        folder=export_folder,  # Root folder in Drive
        fileNamePrefix=f"data/sentinel_downloads/{name.getInfo()}",
        region=geom,
        scale=10,
        maxPixels=1e13
    )
    task.start()
    print("✅ Started export:", name.getInfo())

# ✅ Convert FeatureCollection to list
tile_list = grid_fc.toList(grid_fc.size())
n_tiles = tile_list.size().getInfo()
print(f"Total tiles available: {n_tiles}")

# ✅ Define batch range
start_tile = 0
end_tile = min(start_tile + 100, n_tiles)  # Export 100 tiles at a time

print(f"Starting batch export: tiles {start_tile} to {end_tile-1}")

# ✅ Loop through selected batch
for i in range(start_tile, end_tile):
    export_tile(tile_list.get(i))
    time.sleep(3)

print("Batch export started for 100 tiles.")
print("Monitor progress in https://code.earthengine.google.com → Tasks tab")
print("Files will appear in: Drive → Vaibhavi_Satellite_Assignment/data/sentinel_downloads/")


tried to implement batch processing but not able to. Only exported 100 tiles


In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from shapely.geometry import box, Point
import numpy as np
from scipy import stats

# Define paths
sentinel_tiles_path = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/sentinel_downloads"
land_cover_path = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/raster/land_cover.tif"

print("Tiles folder exists:", os.path.exists(sentinel_tiles_path))
print("Land cover file exists:", os.path.exists(land_cover_path))


In [ ]:
import re
import os
import pandas as pd

sentinel_tiles_path = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/sentinel_downloads"

# all Sentinel-2 tiles
tile_files = [f for f in os.listdir(sentinel_tiles_path) if f.endswith('.tif')]

tile_data = []
pattern = r'tile_([0-9\.\-]+)_([0-9\.\-]+)'

for f in tile_files:
    match = re.search(pattern, f)
    if match:
        # Clean up any trailing dots or invalid characters
        lat_str = match.group(1).rstrip('.')
        lon_str = match.group(2).rstrip('.')
        try:
            lat, lon = float(lat_str), float(lon_str)
            tile_data.append({"filename": f, "lat": lat, "lon": lon})
        except ValueError:
            print(f"Skipping invalid filename: {f}")

# Convert to DataFrame
tile_df = pd.DataFrame(tile_data)
print("✅ Total valid tiles found:", len(tile_df))
tile_df.head()


In [ ]:
import rasterio
from rasterio.windows import Window
import numpy as np
from scipy import stats
import pandas as pd

# Path to land cover raster
land_cover_path = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/raster/land_cover.tif"

# Open the land cover dataset
land_cover = rasterio.open(land_cover_path)
print("✅ Land cover raster loaded successfully!")
print("Raster CRS:", land_cover.crs)
print("Raster resolution:", land_cover.res)
print("Raster shape (height, width):", (land_cover.height, land_cover.width))


In [ ]:
import rasterio
from rasterio.windows import Window
from rasterio.transform import rowcol
import numpy as np
import pandas as pd
import os
import re

# === Path setup ===
land_cover_path = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/raster/land_cover.tif"
sentinel_folder = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/sentinel_downloads"

# Load raster
dataset = rasterio.open(land_cover_path)

# Parse lat/lon from Sentinel filenames
tile_data = []
for f in os.listdir(sentinel_folder):
    match = re.search(r"tile_([0-9\.\-]+)_([0-9\.\-]+)\.tif", f)
    if match:
        lat, lon = float(match.group(1)), float(match.group(2))
        tile_data.append({"filename": f, "lat": lat, "lon": lon})

tiles_df = pd.DataFrame(tile_data)
print("✅ Parsed tiles:", len(tiles_df))

# === Extract 128x128 patches ===
patches_info = []

for i, row in tiles_df.iterrows():
    lat, lon = row["lat"], row["lon"]

    # Convert geographic coords to pixel (row, col)
    r, c = rowcol(dataset.transform, lon, lat)

    # Define window (centered)
    size = 64  # half of 128
    window = Window(c - size, r - size, 128, 128)

    # Read patch
    try:
        patch = dataset.read(1, window=window)
        if patch.shape == (128, 128):
            mode_label = int(np.bincount(patch.flatten()).argmax())  # most frequent class
            patches_info.append({
                "filename": row["filename"],
                "lat": lat,
                "lon": lon,
                "mode_label": mode_label
            })
    except Exception as e:
        print(f"⚠️ Skipped {row['filename']} - out of bounds or error: {e}")

print(f"✅ Extracted {len(patches_info)} valid patches")

# Convert to DataFrame for inspection
labels_df = pd.DataFrame(patches_info)
labels_df.head()





In [ ]:
# === ESA class mapping ===
esa_mapping = {
    10: "Tree cover",
    20: "Shrubland",
    30: "Grassland",
    40: "Cropland",
    50: "Built-up",
    60: "Bare/Sparse vegetation",
    70: "Snow and ice",
    80: "Permanent water bodies",
    90: "Herbaceous wetland",
    95: "Mangroves",
    100: "Moss and lichen",
    255: "No data"
}

# Add standardized labels to DataFrame
labels_df["land_cover_label"] = labels_df["mode_label"].map(esa_mapping)

# Save as CSV for review
output_csv = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/outputs/land_cover_labels.csv"
labels_df.to_csv(output_csv, index=False)

print(f"✅ Saved labels with mapping to: {output_csv}")
labels_df.head()


In [ ]:
import numpy as np
import rasterio
from rasterio.windows import Window
from scipy import stats
import pandas as pd
import os
import re
from tqdm import tqdm

# Re-load tile metadata
tile_dir = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/sentinel_downloads"
land_cover_path = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/data/raster/land_cover.tif"

with rasterio.open(land_cover_path) as src:
    lc_data = src.read(1)
    lc_transform = src.transform
    lc_nodata = src.nodata

tile_data = []
pattern = r"tile_([0-9\.\-]+)_([0-9\.\-]+)"
for f in os.listdir(tile_dir):
    if f.endswith(".tif"):
        match = re.search(pattern, f)
        if match:
            # Clean any stray dots at the end of coordinates
            lat_str = match.group(1).rstrip(".")
            lon_str = match.group(2).rstrip(".")
            lat, lon = float(lat_str), float(lon_str)
            tile_data.append({"filename": f, "lat": lat, "lon": lon})


patch_size = 128  # pixels
half = patch_size // 2

rows = []
for tile in tqdm(tile_data[:500]):  # limit for demo; change to full list later
    lat, lon = tile["lat"], tile["lon"]
    row, col = ~lc_transform * (lon, lat)
    row, col = int(row), int(col)

    with rasterio.open(land_cover_path) as src:
        # Define 128x128 patch window
        window = Window(col - half, row - half, patch_size, patch_size)
        patch = src.read(1, window=window)
        if patch.size == 0:
            continue  # skip out-of-bounds patches

        # Remove nodata values
        valid_pixels = patch[patch != lc_nodata]
        if len(valid_pixels) < patch.size * 0.5:
            continue  # skip patches with >50% nodata

        # Mode and confidence
        mode_value, count = stats.mode(valid_pixels, keepdims=True)
        confidence = count[0] / len(valid_pixels)

        rows.append({
            "filename": tile["filename"],
            "lat": lat,
            "lon": lon,
            "mode_class": int(mode_value[0]),
            "confidence": round(float(confidence), 3)
        })

labels_df = pd.DataFrame(rows)
print(f"✅ Processed {len(labels_df)} patches after filtering edge cases")

# Save result
output_dir = "/content/drive/MyDrive/Vaibhavi_Satellite_Assignment/outputs"
os.makedirs(output_dir, exist_ok=True)
labels_df.to_csv(os.path.join(output_dir, "land_cover_labels_clean.csv"), index=False)
labels_df.head()


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(labels_df, test_size=0.4, random_state=42, stratify=labels_df["mode_class"])

print("✅ Train size:", len(train_df))
print("✅ Test size:", len(test_df))

train_df.to_csv(os.path.join(output_dir, "train_labels.csv"), index=False)
test_df.to_csv(os.path.join(output_dir, "test_labels.csv"), index=False)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
labels_df["mode_class"].value_counts().sort_index().plot(kind="bar")
plt.title("Land Cover Class Distribution")
plt.xlabel("ESA Land Cover Class")
plt.ylabel("Number of Patches")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
